[Reference](https://jnikenoueba.medium.com/fastapi-and-associated-technologies-132b9f389015)

# Associated Technologies
1. Uvicorn
2. Pydantic
3. SQLAlchemy
4. Alembic
5. Tortoise-ORM
6. Motor: An async drive for MongoDB
7. Docker
8. Redis
9. JWT (JSON Web Tokens)
10. OAuth2

In [1]:
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session

DATABASE_URL = "sqlite:///./test.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

class User(Base):
 __tablename__ = "users"
 id = Column(Integer, primary_key=True, index=True)
 name = Column(String, index=True)
 email = Column(String, unique=True, index=True)

class UserCreate(BaseModel):
 name: str
 email: str

app = FastAPI()

def get_db():
 db = SessionLocal()
 try:
   yield db
 finally:
   db.close()

@app.post("/users/", response_model=UserCreate)
def create_user(user: UserCreate, db: Session = Depends(get_db)):
   db_user = User(name=user.name, email=user.email)
   db.add(db_user)
   db.commit()
   db.refresh(db_user)
 return db_user

@app.get("/users/{user_id}", response_model=UserCreate)
def read_user(user_id: int, db: Session = Depends(get_db)):
 db_user = db.query(User).filter(User.id == user_id).first()
 if db_user is None:
   raise HTTPException(status_code=404, detail="User not found")
 return db_user

# Start the application with Uvicorn
# uvicorn main:app - reload